In [ ]:
import cv2
import threading
import winsound 
from ultralytics import YOLO

In [ ]:

# Define the color of the line in BGR format (blue, green, red)
line_color = (0, 255, 0) 

# Define the thickness of the line
line_thickness = 2

# Define the start and end points of the line (x, y)
start_point = (1000, 0)
end_point = (1000, 720)

#Object_Ids
object_Ids =[8]

In [ ]:
def check_line_crossing(object_id, start_point, end_point, direction, results):

    for box in results[0].boxes:
        if box.id.int().cpu() == object_id:

            x, y, width, height = box.xywh.cpu().tolist()[0]

            # Calculate the midpoint of the bounding box
            x_mid = x + width / 2
            y_mid = y + height / 2
            
            # Check if the midpoint crosses the line in the specified direction
            if direction == "left" and x_mid < start_point[0]:
                return True
            elif direction == "right" and x_mid > start_point[0]:
                return True
            elif direction == "up" and y_mid < start_point[1]:
                return True
            elif direction == "down" and y_mid > start_point[1]:
                return True
    
    return False

In [ ]:
def play_alarm():
    # Play the alarm sound for 3 seconds (3000 milliseconds)
    winsound.Beep(1000, 2000)  

# Function to check if object passes through the line in specific direction
def check_and_alert(object_id, direction, results):
    if check_line_crossing(object_id, start_point, end_point, direction, results):
        # Start a new thread to play the alarm sound
        alarm_thread = threading.Thread(target=play_alarm)
        alarm_thread.start()

In [ ]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "test_videos/1.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    # Get the dimensions of the frame
    # height, width, _ = frame.shape
    # print("dimensions: ", height, width, _)


    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        annotated_frame = results[0].plot()

         # Draw the line on the frame
        cv2.line(annotated_frame, start_point, end_point, line_color, line_thickness)

        # Display the annotated frame
        cv2.imshow("YOLOv9 Tracking", annotated_frame)

        # if check_line_crossing(8, start_point, end_point, "right", results):
        check_and_alert(object_Ids[0], "right", results )

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        print("Error: Unable to read the frame.")
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()